In [ ]:
import os 
from dotenv import load_dotenv
load_dotenv()

In [ ]:
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")

In [21]:
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper 
from langchain_openai import AzureChatOpenAI

In [ ]:
llm = Ollama(model="llama3.2:latest", temperature=0)

In [4]:
api_wrapper = ArxivAPIWrapper(top_k_results=2, doc_content_chars_max=500)
arxiv = ArxivQueryRun(api_wrapper=api_wrapper)

In [ ]:
arxiv.invoke('Attention is all you need')

In [ ]:
wikiwrapper = WikipediaAPIWrapper(top_k_results=2, doc_content_chars_max=500)
wiki = WikipediaQueryRun(api_wrapper=wikiwrapper)
wiki.invoke('Artificial Intelligence')

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults

tavily = TavilySearchResults()

In [ ]:
tavily.invoke("Latest advancements in renewable energy technologies")

In [19]:
tools = [arxiv, wiki, tavily]

In [53]:
os.environ['AZURE_OPENAI_API_KEY'] = os.getenv('AZURE_OPENAI_API_KEY')
AZURE_API_ENDPOINT = os.getenv('AZURE_API_ENDPOINT')
AZURE_DEPLOYMENT_NAME = os.getenv('AZURE_DEPLOYMENT_NAME')
AZURE_API_VERSION = os.getenv('AZURE_API_VERSION')
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_PROJECT'] = 'ReAct-Agent'


llm = AzureChatOpenAI(
    model='gpt-4.1-mini',
    azure_endpoint=AZURE_API_ENDPOINT,
    deployment_name=AZURE_DEPLOYMENT_NAME,
    api_version=AZURE_API_VERSION
)

In [ ]:
llm_with_tools = llm.bind_tools(tools)

In [ ]:
response = llm_with_tools.invoke("Explain the concept of transformers in machine learning and provide recent research papers on the topic.")

In [ ]:
response.tool_calls

In [32]:
from IPython.display import display, Image
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.graph.message import add_messages
from langchain_core.messages import AnyMessage
from typing_extensions import TypedDict
from typing import Annotated

In [ ]:
tools

In [103]:
class State(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]

In [111]:
def tool_calling_with_llm(state:State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}
    

In [ ]:
builder = StateGraph(State)
builder.add_node('tool_calling_with_llm', tool_calling_with_llm)
builder.add_node("tools", ToolNode(tools))

builder.add_edge(START, 'tool_calling_with_llm')
builder.add_conditional_edges('tool_calling_with_llm', tools_condition)
builder.add_edge('tools', 'tool_calling_with_llm')

In [113]:
graph = builder.compile()

In [ ]:
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
messages = graph.invoke({'messages': "1706.03762"})

for message in messages['messages']:

    message.pretty_print()

In [56]:
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun, TavilySearchResults
from langchain_community.utilities import ArxivAPIWrapper, WikipediaAPIWrapper


In [57]:
tavily = TavilySearchResults()

In [ ]:
tavily.invoke("Quantum computing breakthroughs in 2024")

In [48]:
api_wrapper = ArxivAPIWrapper(top_k_results=2, doc_content_chars_max=500)
arxiv = ArxivQueryRun(api_wrapper=api_wrapper)

In [49]:
wiki_api_wrapper = WikipediaAPIWrapper(top_k_results=2, doc_content_chars_max=500)
wiki = WikipediaQueryRun(api_wrapper=wiki_api_wrapper)

In [1]:
tools = [arxiv, wiki]

NameError: name 'arxiv' is not defined

In [ ]:
tools

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

In [54]:
def add(a:int, b:int) -> int:
    """
    Adds a and b
    Args:
     a: int
     b: int 
    """
    return a + b

def mul(a:int, b:int) -> int:
    """
    Multiply a and b
    Args:
    a: int 
    b: int 
    """ 
    return a * b 

def div(a:int, b:int) -> float:
    """
    Divide a and b
    Args:
    a: int 
    b: int 
    """ 
    return a / b 

In [59]:
tools = [arxiv, wiki,tavily, add, mul, div]

In [60]:
llm_with_tools = llm.bind_tools(tools)

In [61]:
from pprint import pprint
from langchain_core.messages import AIMessage, HumanMessage

In [ ]:
llm_with_tools.invoke([HumanMessage(content="What is the recent AI News")])

In [65]:
from typing_extensions import TypedDict
from langchain_core.messages import AnyMessage
from typing import Annotated
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode, tools_condition 

In [116]:
class State(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]

In [117]:
def tool_calling_with_llm(state:State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

In [ ]:
builder = StateGraph(State)
builder.add_node('tool_calling_with_llm', tool_calling_with_llm)
builder.add_node("tools", ToolNode(tools))

builder.add_edge(START, 'tool_calling_with_llm')
builder.add_conditional_edges('tool_calling_with_llm', tools_condition)
builder.add_edge('tools', 'tool_calling_with_llm')

In [128]:
from langgraph.checkpoint.memory import MemorySaver
memory = MemorySaver()
graph_with_memory = builder.compile(checkpointer=memory)

In [ ]:
display(Image(graph_with_memory.get_graph().draw_mermaid_png()))

In [145]:
config = {"configurable": {"thread_id": 3}}
messages = [HumanMessage(content="Add 12 and 13")]

In [135]:
message = graph_with_memory.invoke({'messages': messages}, config=config)

In [139]:
message = graph_with_memory.invoke({'messages': [HumanMessage(content='add that to 25')]}, config=config)

In [ ]:
for chunk in graph_with_memory.stream({"messages": [HumanMessage(content="Multiply 5 and 6")]}, config=config, stream_mode="updates"):
    print(chunk)

In [148]:
config = {"configurable": {"thread_id": 5}}

In [ ]:
async for event in graph_with_memory.astream_events({"messages": "My name is KMK and I like playing cricket"}, config=config, version="v2"):
    print(event)
    